In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Define bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# File upload dialog
print("Please select the EEG CSV file:")
Tk().withdraw()  # Prevent the Tkinter root window from appearing
file_name = askopenfilename(filetypes=[("CSV files", "*.csv")])
if not file_name:
    print("No file selected. Exiting.")
    exit()

# Load the CSV file
eeg_data = pd.read_csv(file_name)

# Display the first few rows
print("Data Sample:")
print(eeg_data.head())

# Correct column name for "IsSubmitted" if necessary
if ' IsSubmitted' in eeg_data.columns:
    eeg_data.rename(columns={' IsSubmitted': 'IsSubmitted'}, inplace=True)

# Parse timestamps and calculate sampling frequency
timestamps = pd.to_datetime(eeg_data['Timestamp'], format='%Y.%m.%d.%H.%M.%S.%f')
time_diff = (timestamps.iloc[1] - timestamps.iloc[0]).total_seconds()
fs = int(1 / time_diff)  # Sampling frequency in Hz

# Define epoch duration
epoch_duration = 4  # 2s before + 2s after
epoch_samples = fs * epoch_duration

# Filter unique questions and find the first "CLICKED SUBMIT" for each
questions = eeg_data['Order'].unique()
filtered_epochs = {}

for question in questions:
    # Filter data for the current question
    question_data = eeg_data[eeg_data['Order'] == question]
    # Find the first "CLICKED SUBMIT" event for this question
    submitted_indices = question_data[question_data['IsSubmitted'] == 'CLICKED SUBMIT'].index
    if not submitted_indices.empty:
        idx = submitted_indices[0]  # Take the first occurrence
        start_idx = max(0, idx - 2 * fs)  # 2 seconds before
        end_idx = min(len(eeg_data), idx + 2 * fs)  # 2 seconds after

        # Ensure valid epoch
        if end_idx - start_idx == epoch_samples:
            filtered_epoch = {}
            for channel in ['O1', 'O2', 'T3', 'T4']:
                signal = eeg_data[channel].iloc[start_idx:end_idx].values
                filtered_signal = bandpass_filter(signal, 1, 10, fs)  # 1-10Hz filter
                filtered_epoch[channel] = filtered_signal
            filtered_epochs[question] = (filtered_epoch, question_data['IsCorrect'].iloc[0])  # Add correctness info

# Plot the results for each question
for question, (epoch_data, is_correct) in filtered_epochs.items():
    time_axis = np.linspace(-2, 2, epoch_samples)  # 2 seconds before and after
    correctness = "Correct" if is_correct else "Incorrect"
    plt.figure(figsize=(12, 8))
    for channel, signal in epoch_data.items():
        plt.plot(time_axis, signal, label=channel)
    plt.title(f"Filtered EEG Signal (2s Before and After CLICKED SUBMIT - Question {question})\nAnswer: {correctness}")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Amplitude (mV)")
    plt.legend()
    plt.grid()
    plt.show()
